**nway tutorial**

- Website: https://johannesbuchner.github.io/nway/

In [2]:
from pathlib import Path
from astropy.table import Table
from astropy.io import fits

dir_data = Path("./data")
list(dir_data.iterdir())  # Ensure data files are included in the package

[PosixPath('data/COSMOS_OPTICAL.fits'),
 PosixPath('data/COSMOS_XMM.fits'),
 PosixPath('data/COSMOS_IRAC.fits')]

---
# 命令行模式

```bash
cd /Users/rui/Code/Astronote/38_nway/data
nway.py COSMOS_XMM.fits :pos_err COSMOS_OPTICAL.fits 0.1 --out=example_cmd.fits --radius 15 --prior-completeness 0.9
```

In [8]:
fname = 'example_cmd.fits'
tbl_cmd = Table.read(dir_data / fname)
tbl_cmd

XMM_ID,XMM_RA,XMM_DEC,XMM_pos_err,OPT_ID,OPT_RA,OPT_DEC,OPT_MAG,Separation_OPT_XMM,Separation_max,ncat,dist_bayesfactor,dist_post,p_single,p_any,p_i,match_flag
int32,float64,float64,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float32,float32,int16
1,150.1051483,1.9808168,0.91,-99,-99.0,-99.0,-99.0,--,0.0,1,0.0,1.0,1.0,0.7730111,0.0,0
1,150.1051483,1.9808168,0.91,187581,150.1055997,1.977895,24.89,10.643121,10.643121,2,-18.342627,3.5366116e-29,3.5366116e-29,0.7730111,1.0384995e-29,0
1,150.1051483,1.9808168,0.91,187858,150.1030204,1.9787504,26.0,10.674812,10.674812,2,-18.51767,2.3634341e-29,2.3634341e-29,0.7730111,6.940047e-30,0
1,150.1051483,1.9808168,0.91,187948,150.1065454,1.9779519,23.3,11.473337,11.473337,2,-23.09998,6.183492e-34,6.183492e-34,0.7730111,1.8157361e-34,0
1,150.1051483,1.9808168,0.91,188034,150.105739,1.9781752,23.93,9.744344,9.744344,2,-13.595032,1.9777974e-24,1.9777974e-24,0.7730111,5.8076536e-25,0
1,150.1051483,1.9808168,0.91,188601,150.1036176,1.9795295,22.35,7.197647,7.197647,2,-2.4161048,2.9861275e-13,2.9861275e-13,0.7730111,8.768539e-14,0
1,150.1051483,1.9808168,0.91,188792,150.1084668,1.9815334,25.2,12.214984,12.214984,2,-27.651848,1.7352652e-38,1.7352652e-38,0.7730111,5.095476e-39,0
1,150.1051483,1.9808168,0.91,188965,150.1034074,1.9817002,24.7,7.024619,7.024619,2,-1.7785118,1.2962875e-12,1.2962875e-12,0.7730111,3.8064511e-13,0
1,150.1051483,1.9808168,0.91,189197,150.1072524,1.9819274,23.85,8.561173,8.561173,2,-7.98342,8.087111e-19,8.087111e-19,0.7730111,2.3747195e-19,0


---
# Python模式
- https://johannesbuchner.github.io/nway/nwaylib.html#nwaylib.nway_match

In [17]:
from nwaylib import nway_match
import numpy as np

In [18]:
def table_from_fits(fitsname, poserr_value=None, area=None, magnitude_columns=[]):
	fits_table = fits.open(fitsname)[1]
	table_name = fits_table.name
	ra = fits_table.data['RA']
	dec = fits_table.data['DEC']
	if 'pos_err' in fits_table.data.columns.names:
		poserr = fits_table.data['pos_err']
	else:
		assert poserr_value is not None, ('"pos_err" column not found in file "%s", and no poserr_value passed' % fitsname)
		poserr = poserr_value * np.ones(len(ra))
	if area is None:
		area = fits_table.header['SKYAREA'] * 1.0

	# magnitude columns
	mags = []
	maghists = []
	magnames = []
	#for mag in magnitude_columns:
	for col_name, magfile in magnitude_columns:
		assert col_name in fits_table.data.dtype.names

		mag_all = fits_table.data[col_name]
		# mark -99 as undefined
		mag_all[mag_all == -99] = np.nan

		mags.append(mag_all)
		magnames.append(col_name)
		if magfile == 'auto':
			maghists.append(None)
		else:
			bins_lo, bins_hi, hist_sel, hist_all = np.loadtxt(magfile).transpose()
			maghists.append((bins_lo, bins_hi, hist_sel, hist_all))

	return dict(name=table_name, ra=ra, dec=dec, error=poserr, area=area, mags=mags, maghists=maghists, magnames=magnames)
	# area in square degrees
	# error in arcsec
	# ra/dec in degrees
	# mag: column of something
	# maghists: either (bin, sel, all) tuple or None (for auto)

In [22]:
result = nway_match(
	[
	table_from_fits(dir_data / 'COSMOS_XMM.fits', area=2.0),
	table_from_fits(dir_data / 'COSMOS_OPTICAL.fits', poserr_value=0.1, area=2.0),
	],
	match_radius = 20, # in arcsec
	prior_completeness = 0.9,
)

matching: using fast flat-sky approximation for this match
100%|██████████| 562333/562333 [00:00<00:00, 1238346.47it/s]
matching: collecting from 6979 buckets, creating cartesian products ...
100%|██████████| 6979/6979 [00:00<00:00, 290126.74it/s]
matching: 103094 unique matches from cartesian product. sorting ...
    adding angular separation columns
matching:  37836 matches after filtering by search radius
Primary catalogue "XMM" (1797), density gives 3.71e+07 objects on entire sky
Catalogue "OPT" (560536), density gives 1.16e+10 objects on entire sky
Computing distance-based probabilities ...
    correcting for unrelated associations ...
100%|██████████| 1797/1797 [00:00<00:00, 9737.35it/s]

Computing final probabilities ...
    grouping by primary catalogue ID and flagging ...
/Users/rui/miniconda3/envs/astro/lib/python3.13/site-packages/nwaylib/__init__.py:457: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future vers